# 1. 관절 & 각도 추출

In [1]:
# library
import math
import cv2
import numpy as np
import pandas as pd
from time import time
import mediapipe as mp
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'cv2'

In [ ]:
# load mediapipe pose
mp_pose = mp.solutions.pose

# mediapipe setting
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.3, model_complexity=2)
# MIN_DETECTION_CONFIDENCE : 최소 감지 신뢰도
# MIN_TRACKING_CONFIDENCE : 최소 트랙킹 신뢰도. 이미지 사람없으면 다음 이미지로 자동으로 넘어가서 추적하는데, static_image_mode=True라면 그 기능 무시하고 모든 이미지에서 실행.
# MIN_TRACKING_CONFIDENCE : 값 높게하면 솔루션 견고성 높아지지만 학습시간 길어짐
# model_complexity (0 or 1 or 2) : 랜드마크 정확도와 추론 지연 시간은 일반적으로 모델의 복잡성에 따라 증가

# mediapipe drawing util
mp_drawing = mp.solutions.drawing_utils 

In [ ]:
# define "calculateAngle" model
    # Args :
        ## landmark1 : 1st (x,y,z)
        ## landmark2 : 2nd (x,y,z)
        ## landmark3 : 3rd (x,y,z)
    # Returns :
        ## angle : between 1,2,3
        
def calculateAngle(landmark1, landmark2, landmark3):

    # get landmarks
    x1, y1= landmark1
    x2, y2= landmark2
    x3, y3= landmark3

    # calculatate with math
    # math.atan2(y, x) : 탄젠트의 역함수. atan과 다르게 atan2는 x축으로부터 반시계방향으로 각도를 계산. 이후 라디안 값으로 뱉어냄
    # math.degrees : 라디안값을 각도로 변경해주는 함수
    angle = math.degrees(math.atan2(y3 - y2, x3 - x2) - math.atan2(y1 - y2, x1 - x2))
    
    # find opposite angle
    if angle < 0.0:
        angle += 360
    if angle >180.0:
        angle = 360.0 - angle
  
    # iterate
    return angle

In [ ]:
# 두 측정점과 지면과의 각도 구하는 함수

def calculateAngle2(landmark1, landmark2):

    # get landmarks
    x1, y1= landmark1
    x2, y2= landmark2
    x3 = x2
    y3 = y2 + 1
    
    # calculate with math
    angle = math.degrees(math.atan2(y3 - y2, x3 - x2) - math.atan2(y1 - y2, x1 - x2))
    
    # find opposite angle
    if angle < 0.0:
        angle += 360
    if angle >180.0:
        angle = 360.0 - angle
  
    # iterate
    return angle

In [ ]:
# calculateAngle 함수와 기능은 같지만 다르게 짠 코드

def calculateAngle3(landmark1, landmark2, landmark3):
    x1, y1= landmark1
    x2, y2= landmark2
    x3, y3= landmark3
    
    # calculate angle
    radians = np.arctan2(y3-y2, x3-x2) - np.arctan2(y1-y2, x1-x2)
    angle = np.abs(radians*180.0/np.pi)
    
    if angle < 0.0:
        angle +=360
    
    if angle >180.0:
        angle = 360-angle
        
    return angle

In [ ]:
total = []

# 이미지 수 만큼 반복수 설정
for i in range(4):
    # read image
    sample_img = cv2.imread('C:\\Users\\user\\graduation_work\\data\\sample'+str(i)+'.jpg')
    
    # pose detection (RGB). 이미지에서 스켈레톤 구조 추정하는 단계
    results = pose.process(cv2.cvtColor(sample_img, cv2.COLOR_BGR2RGB))
    
    # 추정한 스켈레톤 구조의 측정값(관절 x,y값)들 landmarks 변수의 저장
    landmarks = results.pose_landmarks.landmark
    
    # 추정한 좌표 값의 신뢰도가 지정한 신뢰도보다 낮을 경우 그 이미지는 패스하고 다음 이미지로 넘어감
    if landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].visibility < 0.7:
        print(i,'번째 이미지의 right_hip 신뢰도가 0.7보다 낮습니다'); continue
    if landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].visibility < 0.7:
        print(i,'번째 이미지의 right_knee 신뢰도가 0.7보다 낮습니다'); continue
    if landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].visibility < 0.7:
        print(i,'번째 이미지의 right_ankle 신뢰도가 0.7보다 낮습니다'); continue
    if landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].visibility < 0.7:
        print(i,'번째 이미지의 right_shoulder 신뢰도가 0.7보다 낮습니다'); continue
    if landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].visibility < 0.7:
        print(i,'번째 이미지의 right_foot_index 신뢰도가 0.7보다 낮습니다'); continue

    if landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].visibility < 0.7:
        print(i,'번째 이미지의 left_hip 신뢰도가 0.7보다 낮습니다'); continue
    if landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].visibility < 0.7:
        print(i,'번째 이미지의 left_knee 신뢰도가 0.7보다 낮습니다'); continue
    if landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].visibility < 0.7:
        print(i,'번째 이미지의 left_ankle 신뢰도가 0.7보다 낮습니다'); continue
    if landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility < 0.7:
        print(i,'번째 이미지의 left_shoulder 신뢰도가 0.7보다 낮습니다'); continue
    if landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].visibility < 0.7:
        print(i,'번째 이미지의 left_foot_index 신뢰도가 0.7보다 낮습니다'); continue
    
    # squat 모델학습에 사용할 관절좌표 x,y 값만 리스트로 따로 구분하여 저장
    right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
    right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
    right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
    right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
    right_foot_index = [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y]

    left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
    left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
    left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
    left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
    left_foot_index = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x, landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]

    # squat 모델학습에 사용할 각도변수 생성 (총 6개 변수, 관절 x,y 좌표값을 이용하여 각도 계산한 것)
    angle_right_knee = calculateAngle(right_hip, right_knee, right_ankle)
    angle_left_knee = calculateAngle(left_hip, left_knee, left_ankle)
    angle_right_hip = calculateAngle(right_shoulder, right_hip, right_knee)
    angle_left_hip = calculateAngle(left_shoulder, left_hip, left_knee)
    angle_right_ankle = calculateAngle(right_knee, right_ankle, right_foot_index)
    angle_left_ankle = calculateAngle(left_knee, left_ankle, left_foot_index)
    
    # 생성한 변수들 하나의 리스트로 묶어서 저장
    angle_list = []
    angle_list.append(angle_right_knee)
    angle_list.append(angle_left_knee)
    angle_list.append(angle_right_hip)
    angle_list.append(angle_left_hip)
    angle_list.append(angle_right_ankle)
    angle_list.append(angle_left_ankle)
    
    
    # 이미지 시각화
    if results.pose_landmarks:
    
        # draw skeleton
        mp_drawing.draw_landmarks(image=sample_img, landmark_list=results.pose_landmarks, connections=mp_pose.POSE_CONNECTIONS)
       
        # plot on mat
        fig = plt.figure(figsize = [10, 10])
        plt.title("Output");plt.axis('off');plt.imshow(sample_img[:,:,::-1]);plt.show()
    
    total.append(angle_list)

error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [ ]:
total

[]

In [ ]:
total.shape

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
# list를 dataframe로 변경
df_angle = pd.DataFrame(total, columns=['right_knee', 'left_knee', 'right_hip', 'left_hip', 'right_ankle', 'left_ankle'])

In [ ]:
df_angle

,right_knee,left_knee,right_hip,left_hip,right_ankle,left_ankle
0,86.201020,161.788648,106.377992,139.080494,77.133831,61.784404
1,165.170225,130.428959,157.385623,178.541860,167.327045,147.136870


In [ ]:
# 엑셀로 저장
df_angle.to_excel('C:\\Users\\user\graduation_work\\Squat Land Mark3.xlsx', index=False, sheet_name='squat')

In [ ]:
# csv 파일로 저장
df_angle.to_csv('C:\\Users\\user\\graduation_work\\Squat Land Mark2.csv', index=False)